# Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
RUN_DIR = 'runs/0/'
df = pd.concat([pd.read_parquet(RUN_DIR+f) for f in os.listdir(RUN_DIR)])
len(df)

In [ ]:
df.sample(3).T

# Pre-process

### Get item counts

In [ ]:
item_cols = [col for col in df.columns if 'items' in col]
item_cols

In [ ]:
item_types = df[df['after']][item_cols[0]].dropna().unique()
item_types

In [ ]:
for i, item_type in enumerate(item_types):
    df[item_type]  = (df[item_cols] == item_types[i]).sum(axis=1)

In [ ]:
df[['attempt_id', 'after']+item_types.tolist()]

### Get trophy value

In [ ]:
trophy_cols = [col for col in df.columns if 'trophies' in col]
trophy_cols

In [ ]:
df['trophy_point'] = df[trophy_cols].fillna(0).sum(axis=1)

In [ ]:
df[['adventure_id', 'after', 'succeeded', 'trophy_point']]

# Get values

In [ ]:
import json

In [ ]:
with open('values.json') as f:
    values = json.load(f)
    
# values = pd.DataFrame(values, index=range(1)).T.reset_index().rename(columns={'index': 'attribute', 0: 'value'})
values = pd.DataFrame(values, index=range(1))
values.columns = [col + '.value' for col in values.columns]
values

# Calculate values

In [ ]:
df = df.merge(values, how='cross')

In [ ]:
df['value'] = (
    # Victory track
    (df['ancient_one.elder_signs'] * df['ancient_one.elder_sign.value'])
    + (df['ancient_one.doom_tokens'] * df['ancient_one.doom_token.value'])
    # Character stats
    + (df['characters[0].health'] * df['character.health.value'])
    + (df['characters[0].sanity'] * df['character.sanity.value'])
    + (df['characters[0].blessed'] * df['character.blessed.value'])
    + (df['characters[0].cursed'] * df['character.cursed.value'])
    # Items
    + (df['CommonItem'] * df['character.CommonItem.value'])
    + (df['UniqueItem'] * df['character.UniqueItem.value'])
    + (df['Spell'] * df['character.Spell.value'])
    + (df['Ally'] * df['character.Ally.value'])
    + (df['Clue'] * df['character.Clue.value'])
    + (df['trophy_point'] * df['trophy_point.value'])
    # Other
    + (df['num_monsters'] * df['monster.value'])
    + (df['clock.hour'] * df['clock.turn.value'] / 3)
)

### Expected value of success vs failure

In [ ]:
adventures = df[df['after']].groupby(['adventure_id', 'scenario.name', 'succeeded'])\
    .agg({'value': 'mean'})\
    .reset_index()

adventures = pd.pivot_table(adventures, index=['adventure_id', 'scenario.name'], columns=['succeeded']).reset_index()
adventures.columns = ['adventure_id', 'scenario.name', 'failed.value.mean', 'succeeded.value.mean']
adventures

### Success probabilities

In [ ]:
df['count'] = 1
success_rates = df[df['after']].groupby(['adventure_id', 'scenario.name'])\
    .agg({'succeeded': 'sum', 'count': 'sum'})\
    .reset_index()

success_rates['success_probability'] = success_rates['succeeded'] / success_rates['count']
success_rates

In [ ]:
adventures = adventures.merge(success_rates, on=['adventure_id', 'scenario.name'])
# adventures

In [ ]:
# This is always the same because the dummy game setup is identitical in each trial
baseline_value = df[~df['after']]['value'].unique().item()

In [ ]:
adventures['expected_value'] = (
    adventures['success_probability'] * adventures['succeeded.value.mean'] +
    (1 - adventures['success_probability']) * adventures['failed.value.mean']
)
adventures['expected_return'] = adventures['expected_value'] - baseline_value

In [ ]:
adventures[['adventure_id', 'scenario.name', 'succeeded.value.mean', 'failed.value.mean', 'success_probability', 'expected_return']]